# TORONTO NEIGHBORHOOD SCRAPE, SEGMENT, CLUSTER CAPSTONE
### explore, segment, and cluster the neighborhoods in the city of Toronto.
1. Create structured dataframe of Toronto data
   * to explore and cluster the neighborhoods in Toronto scrape https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M . 
        * ignore neighborhoods/postal codes that are not assigned a borough.
        * boroughs lacking a neighborhood have the same name.
   * read data into a pandas dataframe with column keys: Postal Code, Borough, Neighborhood.
        * Postal Code serves as the primary key
        * Neighborhoods with same Postal Code will be concatenated into a single value: "N1,N2"
2. Comment code and provide Markdown titles.
3. Last line display dataframe and should give shape of cleaned dataframe. 
4. Make this the first notebook. Publish to github in an open access directory, provide link to github ipynb for first submission.
5. *Change name of notebook to xxx-v1.ipynb
6. Use Geocoder Python package: https://geocoder.readthedocs.io/index.html to get mapping data. 
    * Final output should be table with column keys: postal code, borough, neighborhood, latitude, longitude
    * May want to incorporate code:
                import geocoder # import geocoder
                lat_lng_coords = None
                while(lat_lng_coords is None):
                  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
                  lat_lng_coords = g.latlng
                latitude = lat_lng_coords[0]
                longitude = lat_lng_coords[1] 
7. Verify latitude and longitude http://cocl.us/Geospatial_data
8. Publish to Github, in an open access directory, provide link to github ipynb file for final submission.
9. *Change name to xxx-v2.ipynb
10. Explore and cluster the neighborhoods in Toronto. Use only boroughs that contain the word Toronto and replicate the same analysis as done with New York City data. 
    * use foursquare to identify categories of venues.
    * use folium to generate maps to visualize your neighborhoods and how they cluster together. 
11. Publish to Github, in an open access directory, provide link to github ipynb file for final submission.

## OVERVIEW
1. Load Relevant Libraries
2. Scrape wikipedia website.
3. Clean Toronto Data. 
4. Deliverable 1: Cleaned Data Table
5. Get postal code latitude and longitude. Display geolocated pandas data.
6. Explore and cluster toronto neighborhoods using NYC analysis as template.

### 1. LOAD RELEVANT LIBRARIES
1. General processing
2. Scraping
3. HTML and image display
4. geolocation
5. mapping
6. clustering

In [1]:
#1. load General processing libraries
import pandas as pd 
import numpy as np 
import random 

#2. load libraries for scraping
from bs4 import BeautifulSoup
import requests # library to handle requests

#3. libraries for displaying HTML and images
from IPython.display import Image 
from IPython.core.display import display, HTML

#4. loading libraries for geolocation
# module to convert an address into latitude and longitude values
try:
    from geopy.geocoders import Nominatim
except:
    !conda install -c conda-forge geopy --yes 
    from geopy.geocoders import Nominatim

#5. loading libraries for Mapping    
try:
    import folium # plotting library
except:
    !conda install -c conda-forge folium=0.5.0 --yes
    import folium # plotting library

#6. Loading libraries for clustering
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize



### 2. SCRAPING WIKIPEDIA DATA
1. Load wikipedia page
2. Extract table
3. format HTML table into string type and read data into pd.DataFrame
4. Display wrangled data

In [2]:
#1. Load wikipedia page
# use requests.get to read in wikipedia list of toronto postal codes.
html_url =  'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_file = requests.get(html_url)

#2. Extract table
#structure website using beautiful soup
file = BeautifulSoup(html_file.content, 'lxml')
#Extract postal data table (review website coding to identify data is located in a table structure)
table = file.table

#3. format HTML table into string type
#cast postal data table html as string so it can be read by pandas read_html.
toronto_raw_postal_wiki_df = pd.read_html(str(table))[0]

#4. Display wrangled data
#verify raw postal data table
print("Raw dataframe contains {} postal codes".format(toronto_raw_postal_wiki_df.shape[0]))
toronto_raw_postal_wiki_df.head()

Raw dataframe contains 288 postal codes


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### 3. CLEAN TORONTO DATA
1. identify and eliminate rows with Borough value "Not assigned"
2. assign Neighborhood value to Borough value if Neighborhood is "Not Assigned"
3. group by postcode. If there are multiple neighborhoods, replace with comma delimited string.
4. Display cleaned table, show count of rows and columns.

In [3]:
#1. identify and eliminate rows with Borough value "Not assigned"
#Make logical index of rows where borough is notassigned
Bad_Borough_Index = toronto_raw_postal_wiki_df.loc[:,'Borough'].isin(['Not assigned'])
#Use logical negation to drop bad rows. Assign values optimistically to new clean table.
toronto_postal_df = toronto_raw_postal_wiki_df.loc[~Bad_Borough_Index,:]

#2. assign Neighborhood value to Borough value if Neighborhood is "Not Assigned"
#Find rows where Neighborhood is not assigned.
Bad_Neighborhood_Index = toronto_postal_df.loc[:,'Neighbourhood'].isin(['Not assigned'])
print(" {} rows were found to have missing Neighborhood values".format(sum(Bad_Neighborhood_Index)))
print()        
#Assign Borough value to Bad Neighborhood values.
toronto_postal_df.loc[Bad_Neighborhood_Index,'Neighbourhood']=toronto_postal_df.loc[Bad_Neighborhood_Index,'Borough']
#check data frame for Missing Neighborhood values.
Bad_Neighborhood_Index2 = toronto_postal_df.loc[:,'Neighbourhood'].isin(['Not assigned'])
print("After correction, {} rows were found to have missing Neighborhood values".format(sum(Bad_Neighborhood_Index2)))
toronto_postal_df = toronto_postal_df.sort_values(by=['Postcode']).reset_index(drop=True)
toronto_postal_df.head()

 1 rows were found to have missing Neighborhood values

After correction, 0 rows were found to have missing Neighborhood values


/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Rouge
1,M1B,Scarborough,Malvern
2,M1C,Scarborough,Port Union
3,M1C,Scarborough,Rouge Hill
4,M1C,Scarborough,Highland Creek


In [4]:
#3. group by postcode. If there are multiple neighborhoods, replace with comma delimited string.
#group by postcode, replace neighborhoods with list.
toronto_clean_df = pd.DataFrame(toronto_postal_df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(list))
toronto_clean_df.reset_index(inplace=True)  
toronto_clean_df['Neighbourhood'] = toronto_clean_df.Neighbourhood.apply(lambda x: ", ".join(x)).tolist()

#4. Display cleaned table, show count of rows and columns.
HTML(toronto_clean_df.to_html())

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Golden Mile, Oakridge, Clairlea"
8,M1M,Scarborough,"Cliffcrest, Scarborough Village West, Cliffside"
9,M1N,Scarborough,"Cliffside West, Birch Cliff"


In [5]:
print("The shape is {} rows (Postcodes) and {} columns (Pstcd-Brgh-Nghbrhd)".format(toronto_clean_df.shape[0],toronto_clean_df.shape[1]))

The shape is 103 rows (Postcodes) and 3 columns (Pstcd-Brgh-Nghbrhd)
